#EduWeaver

EduWeaver uses ChatGPT API to make a LiaScript course. EduWeaver can be considered an AutoCourse maker.

- Specify a topic and a few other parameters (i.e. sub-sections to nclude and exclude)
- Creates a list of sub sections (or chapters)
- Generates the content
- Generates appropriate activities eg MCQ's
- Adds simplified sections to explain complex things better
- Includes external links or readings
- Includes code snippets/examples for programming related topics
- Saves as a Markdown file for valid [LiaScript](https://liascript.github.io/)

Coming soon:

- Additional interactive learning activities
- Other export formats eg HTML and SCORM

Eduweaver uses LangChain and the latest LangChain Expression Language (LCEL) syntax.

## Setup

Enter a few details about the course and your OpenAI API key. You can create a new API key by going to your OpenAI Account [https://platform.openai.com/account/api-keys].

In [4]:
#@title Course Generation Settings
topic = 'Design Thinking'  #@param {type: "string"}
teaching_method = 'Explain concepts is a simple and straight forward manner'  #@param {type: "string"}
no_sub_topics = 8  #@param {type: "slider", min: 2, max: 10}
sub_topics_to_include = "Idea integration" #@param {type: "string"}
sub_topics_to_not_include = "" #@param {type: "string"}
audience = 'Educators'   #@param {type: "string"}
no_quiz_questions_in_sub_topic = 5 #@param {type: "slider", min: 1, max: 6}
narrator_language = "Australian Female" #@param ["UK English Female", "US English Female", "Australian Female", "UK English Male", "US English Male", "Australian Male"]
output_filename = 'DesignThinking_Course.md'  #@param {type: "string"}

#@markdown ---


In [ ]:
# Set your environment variables
# Don't include quotes around the key
%env OPENAI_API_KEY=

In [6]:
# Install required libraries
!pip install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all th

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

# Setup the model
model = ChatOpenAI(temperature=0.7, model_name='gpt-4')
json_output_parser = SimpleJsonOutputParser()
string_output_parser = StrOutputParser()

# Get TOC sections for the course

print("- Getting Course TOC.")

# First extract the additional specified sub-sections
list_of_additional_sub_sections = sub_topics_to_include.split(';')
sub_topics_to_include_text = " "
if len(list_of_additional_sub_sections) > 0:
  sub_topics_to_include_text = "In the sub sections also please include these: " + sub_topics_to_include

list_of_sub_topics_not_to_incude = sub_topics_to_not_include.split(';')
sub_topics_to_not_include_text = " "
if len(list_of_sub_topics_not_to_incude) > 0:
  sub_topics_to_not_include_text = "Please don't include content on these sub-topics (THIS IS VERY IMPORTANT): " + sub_topics_to_not_include


if teaching_method!="":
  teaching_method_to_include_text = "Use this teaching and writing style: " + teaching_method


act_as_ld_template='''
                      Please act as a subject matter expert on the topic of {topic}. You also have learning designer skills and knowledge of markdown and json.
                      You know how to design online modules with a deep knowledge of the sections that need to be included for the topic. You can break up the content into chunks, explain concepts so that can easily be understood, write quizzes to test knowledge and write creative learning activities for students.
                   '''
ask_for_sections_template='''
                              Could you please come up with {no_sub_topics} sections for an online module on the topic of {topic} for {audience} in json format?
                              {sub_topics_to_include_text}
                              {sub_topics_to_not_include_text}
                              Here is an example of the format you need to return:
                              [{{"section_name": "The name of the section 1", "section_description": "The descriptions of the section 1"}}]
                              Note that you must only return the section_name and section_description fields in the json that is returned.
                              Please don't return any intro text before or after the json. YOU MUST RETURN ONLY VALID JSON.
                          '''

toc_prompt = ChatPromptTemplate.from_template(act_as_ld_template + ask_for_sections_template)

toc_chain = toc_prompt | model | json_output_parser

sections = toc_chain.invoke({"topic": topic, "no_sub_topics": no_sub_topics, "sub_topics_to_include_text": sub_topics_to_include_text, "sub_topics_to_not_include_text": sub_topics_to_not_include_text, "audience": audience })

updated_sections = []
for section in sections:
  new_section = {"topic": topic, "no_sub_topics": no_sub_topics, "sub_topics_to_include_text": sub_topics_to_include_text, "sub_topics_to_not_include_text": sub_topics_to_not_include_text, "no_quiz_questions_in_sub_topic":no_quiz_questions_in_sub_topic, "audience": audience, "teaching_method_to_include_text": teaching_method_to_include_text, "section_name": section["section_name"], "section_description": section["section_description"]}
  updated_sections.append(new_section)

# Get content for each sub section

print("- Getting content for each sub section.")

section_details_template='''
                              Could you please write the content for this section name: {section_name}
                              The content must be appropriate for this audience: {audience}
                              The description of the topic that you must write is {section_description}.
                              {teaching_method_to_include_text}
                              {sub_topics_to_not_include_text}
                              Please first output the section name as a level 2 markdown eg ## {section_name}
                              If the section is on programming or coding please include code examples within backticks and specify the language and filename eg:
                              ``` js     +Filename.js
                                  let hi = "Hello World"
                              ```
                              If the section needs math just place the Latex for the math equations between $ signs eg $ \frac{{a}}{{\sum{{b+i}}}} $
                              You can include sub sections. Please place the sub section headings within ** and ** followed by \n (eg **Goals** \n ) as this is the markdown for sub sections at heading level 3 and a newline is required after.
                              For difficult concept please include a section also "**Additional Links**" with links to Wikipedia and please make sure the proper markdown is used for links. Only include links to wikipedia that exist, don't include links to other sites (this is very important).
                          '''

author_sections_prompt = ChatPromptTemplate.from_template(act_as_ld_template + section_details_template)

author_sections_chain = author_sections_prompt | model | string_output_parser

sections_markdown = author_sections_chain.batch(updated_sections)

# Get creative projects

print("- Getting creative projects.")

creative_project_template='''
                              Could you please come up with a few creative project ideas for {topic} that will be appropriate for {audience}?
                              Be creative and the project should assist students in learning higher level skills that can't be obtained by doing multiple choice tests.
                              The high level project ideas should develop levels from Blooms taxonomy including Application, Analysis, Synthesis, and Evaluation.
                              Please don't output the section name or the type of level used from Blooms taxonomy but do include an introductory paragraph.
                              {sub_topics_to_not_include_text}
                              If the project for the section is on programming or coding please include code examples for the project within backticks and specify the language and filename eg:
                              ``` js     +Filename.js
                                  let hi = "Hello World"
                              ```
                              If the project for the section needs maths please place the Latex for the math equations between $ signs eg $ \frac{{a}}{{\sum{{b+i}}}} $
                              Don't include quiz questions.
                              Include each project as a sub sections in markdown. Please place the sub section headings within ** and ** followed by \n (eg **Project 1** \n ) as this is the markdown for sub sections at heading level 3 and a newline is required after.
                          '''

creative_project_prompt = ChatPromptTemplate.from_template(act_as_ld_template + creative_project_template)

creative_project_chain = creative_project_prompt | model | string_output_parser

creative_projects = creative_project_chain.invoke({"topic": topic, "no_sub_topics": no_sub_topics, "sub_topics_to_include_text": sub_topics_to_include_text, "sub_topics_to_not_include_text": sub_topics_to_not_include_text, "audience": audience })

# Get Quiz Questions for each section

print("- Getting quiz questions for each sub section.")

section_assessment_template='''
                              Could you please write quiz questions for the content in {section_name} that will be appropriate for {audience}?
                              The description of the topic that you must write quiz questions for is {section_description}.
                              Please write {no_quiz_questions_in_sub_topic} questions in the json format that is given in the example below:

                              [
                                {{"question": "Question 1",
                                 "question_type": "single_option",
                                 "options": [{{"option_name": "Option 1", "correct": "false"}}, {{"option_name": "Option 2", "correct": "true"}}]
                                }},
                                {{"question": "Question 2",
                                 "question_type": "multiple_options",
                                 "options": [{{"option_name": "Option 1", "correct": "false"}}, {{"option_name": "Option 2", "correct": "true"}}, {{"option_name": "Option 3", "correct": "true"}} the]
                                }}
                              ]

                              The questions must be returned in the same json format as what is given above. The incorrect answers should not be obvious or easy.
                              Please note that you there are 2 types of questions namely single_option and multiple_options.
                          '''

assessment_prompt = ChatPromptTemplate.from_template(act_as_ld_template + section_assessment_template)

assessment_chain = assessment_prompt | model | json_output_parser

quiz_questions = assessment_chain.batch(updated_sections)



- Getting Course TOC.
- Getting content for each sub section.
- Getting creative projects.
- Getting quiz questions for each sub section.


In [8]:
# Convert Quiz Json to LiaScript Markdown
print("- Convert MCQ Json to Liascript Markdown.")

quiz_questions = [[] if x is None else x for x in quiz_questions] # Makes None an empty list

course_assessment_markdown = {}

for index, quiz_section in enumerate(quiz_questions):
  question_markdown = ""
  #print(quiz_section)
  for question in quiz_section:
    question_title = question['question']
    question_type = question['question_type']
    if (question_type == "single_option"):
      open_brace = '('
      close_brace = ')'
    else:
      open_brace = '['
      close_brace = ']'
    question_markdown += question_title + '\n\n'
    options = question['options']
    for option in options:
      option_name = option["option_name"]
      if option['correct'] == "true":
        marker = 'X'
      else:
        marker = ' '
      question_markdown += f'''    [{open_brace}{marker}{close_brace}] {option_name} \n'''
    question_markdown += '\n'
    section_name = sections[index]['section_name']
    course_assessment_markdown[section_name] = question_markdown


# Collate course and force download of the Markdown file

print("- Collating course content.")

course_markdown = ""

course_markdown += f"""
<!--

author:   EduWeaver - An AutoCourse Creator Using ChatGPT
email:    nobody@nowhere.com
version:  0.0.2
language: en
narrator: {narrator_language}

logo:     https://liascript.github.io/img/bg-showcase-1.jpg

comment:  Eduweaver generates course content using chatGPT and outputs in Liascript Markdown

-->
"""

course_markdown += f'''# {topic}\n'''

# Make the TOC
toc = "In this course the following content will be covered: \n\n"
for section in sections:
  section_name = section['section_name']
  section_description = section['section_description']
  toc += f'''- {section_name} \n {section_description} \n\n'''

course_markdown += """
> This course is completely generated by Eduweaver (using ChatGPT) in Liascript Markdown format.
> Please verify the content before publishing the course. \n \n
"""

course_markdown += toc

for index, section in enumerate(sections):
  section_name = section['section_name']
  section_description = section['section_description']
  section_content = sections_markdown[index]
  if section_name in course_assessment_markdown:
    section_assessment = course_assessment_markdown[section_name]
  else:
    section_assessment = None

  course_markdown += f'''{section_content}\n\n'''
  if section_assessment!=None:
    course_markdown += f'''### Quiz Questions \n\n'''
    course_markdown += f'''{section_assessment}\n'''

course_markdown += f'''## Project Ideas \n\n'''
course_markdown += f'''{creative_projects}\n\n'''

with open(output_filename, 'w') as writefile:
    writefile.write(course_markdown)

print("- Forcing file download.")
from google.colab import files
files.download(output_filename)

print("- If the file does not download, click on the Folder on the left and you will be able to download manually.")

- Convert MCQ Json to Liascript Markdown.
- Collating course content.
- Forcing file download.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- If the file does not download, click on the Folder on the left and you will be able to download manually.


## What to do next?

- Place the markdown (.md) file on github and then view using the [Liascript Viewer](https://liascript.github.io/)
OR
- Copy the markdown content into the [Liascript LiveEditor](https://liascript.github.io/LiveEditor/) for editing and previewing.